# Data Science capstone week 3
## section one

In [2]:
import pandas as pd
import numpy as np
from unicodedata import normalize
import matplotlib.pyplot as plt
from pathlib import Path
import requests
from pandas.io.json import json_normalize
import pandas.io.json
import json

%matplotlib inline

In [2]:
dfTRT = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
len(dfTRT)

3

In [3]:
dfTRT = dfTRT[0]
dfTRT

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
dfTRT.groupby(by = 'Borough').agg('count')

,Postal Code,Neighbourhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Not assigned,77,77
Scarborough,17,17


In [5]:
rows = len(dfTRT.index)
dropped = 0
for i in range (0, rows):
    if dfTRT.at[i, 'Borough'] == 'Not assigned':
        dfTRT.drop(i, inplace = True)
        rows = rows - 1
        dropped = dropped +1
print('Rows Remaining: ', len(dfTRT.index), 'Rows Dropped: ', dropped)
dfTRT.reset_index(inplace = True)

Rows Remaining:  103 Rows Dropped:  77


In [6]:

for i in range (0, rows):
    if dfTRT.at[i, 'Neighbourhood'] == 'Not assigned':
       
        dfTRT.at[i, 'Neighbourhood'] = dfTRT.at[i, 'Borough'] 
   
    


In [7]:
dfTRT.drop('index',axis = 1)



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
p = Path('~/Documents/Technical/Geospatial_Coordinates.csv')

dfCOOR = pd.read_csv(p, low_memory=False)
dfCOOR

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
dfM1 = dfCOOR.set_index('Postal Code')
dfM2 = dfTRT.set_index('Postal Code')

dfNew = dfM1.join(dfM2)



#dfFin = dfNew[['Postal Code'],['Latitude'], ['Longitude'], ['Borough'], ['Neighbourhood']]
#dfFin

dfNew.drop(['index'],inplace = True, axis = 1)

dfNew

,Latitude,Longitude,Borough,Neighbourhood
Postal Code,,,,
M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
M1G,43.770992,-79.216917,Scarborough,Woburn
M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...
M9N,43.706876,-79.518188,York,Weston
M9P,43.696319,-79.532242,Etobicoke,Westmount
M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [10]:
!pip install folium --user
import folium.PolyLine()

In [11]:
import folium

In [12]:
TRTmap = folium.Map(location = [43.7, -79.5], zoom_start = 10)
TRTmap

In [13]:
for lat ,long, bor, neigh in zip(dfNew['Latitude'], dfNew['Longitude'], dfNew['Borough'], dfNew['Neighbourhood']):
    label = '{}, {}'.format(bor, neigh)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, long], 
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = .6,
    parse_html = False
    ).add_to(TRTmap)
TRTmap

In [14]:
import json

In [15]:
#I want to easily iterate through rows, so I'll reset the index with nice sequential numbers
dfNew.reset_index(inplace= True)
dfNew

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [16]:
def get_category_type(row):
    try:
        categoriesList = row['categories']
    
    except:
        categoriesList = row['venue.categories']
    if len(categoriesList) == 0:
        return 0
    else:
        return categoriesList[0]['name']

In [17]:
venueList = []
for i in range(len(dfNew.index)):
    #request 5 food venues per neighborhood
    
    client_id = 'KGLEGNEJRVPLNRZSV4KSNR3PNZCBWSIJXPU151PTMCFIC1JL'
    client_secret = 'YLBMIBUMMP2A50O5GBEVYBEPA21D0NG3RUQNI3TOW2EM3SIQ'
    version = '20200605'
    
    foodCode = '4d4b7105d754a06374d81259'
    lim = '10'
    rad = '500'
    nlat = dfNew.loc[i, 'Latitude']
    nlon = dfNew.loc[i, 'Longitude']
    surl = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(client_id, client_secret, version, nlat, nlon, rad, lim, foodCode)
    rets = requests.get(surl).json()
    
    #cleans data for pandas dataframe
    Venues= rets['response']['groups'][0]['items']
    dfVenues = json_normalize(Venues)

    dfilter = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
    try: dfVenues = dfVenues[dfilter]
    except: continue
    dfVenues['venue.categories'] = dfVenues.apply(get_category_type, axis =1 )
    for j in range (len(dfVenues.index)):
        venueList.append(
        [dfNew.loc[i, 'Neighbourhood'], nlat, nlon, dfVenues.loc[j, 'venue.name'], dfVenues.loc[j,'venue.categories'], dfVenues.loc[j, 'venue.location.lat'], dfVenues.loc[j,'venue.location.lng']])
    
dfFinal = pd.DataFrame(venueList, columns = ['Neighbourhood', 'Nlat', 'Nlon', 'venue name', 'venue type', 'vlat', 'vlon'])

    

In [44]:
rets

{'meta': {'code': 200, 'requestId': '6000e07e3253c51e9a622849'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'food',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.711248304499996,
    'lng': -79.58784096674265},
   'sw': {'lat': 43.70224829549999, 'lng': -79.60026783325736}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [18]:
#before I use onehot encoding to deal with the categories, lets see how many columns that will add
len(pd.unique(dfFinal['venue type']))

66

In [19]:
#66 unique values, a lot, but not an impossible amount ill just go for it


OneHot = pd.get_dummies(dfFinal['venue type'])
OneHot



,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,...,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
545,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
546,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:


M = dfFinal.groupby('Neighbourhood')['venue name'].sum() 


In [21]:
dfReady = dfFinal.join(OneHot)
dfReady

,Neighbourhood,Nlat,Nlon,venue name,venue type,vlat,vlon,American Restaurant,Asian Restaurant,BBQ Joint,...,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,Fast Food Restaurant,43.807448,-79.199056,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",43.806686,-79.194353,Meena's Fine Foods,Indian Restaurant,43.804476,-79.199753,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Doug's Bakery,Bakery,43.762400,-79.192285,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,Restaurant,43.765951,-79.191275,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,Subway,Sandwich Place,43.742645,-79.589643,0,0,0,...,0,0,0,0,0,0,0,0,0,0
544,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545,0,0,0,...,0,0,0,0,0,0,0,0,0,0
545,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,Pizza Pizza,Pizza Place,43.741569,-79.584489,0,0,0,...,0,0,0,0,0,0,0,0,0,0
546,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,Pizza Nova,Pizza Place,43.736761,-79.589817,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:

dfNeigh = dfReady.groupby(['Neighbourhood']).sum()

In [23]:
dfCl = dfNeigh.drop(['Nlat','Nlon','vlat', 'vlon'], axis = 1)

In [24]:
dfCl.reset_index(inplace = True)

In [25]:

latlist=[]
lonlist=[]
for i in range (len(dfCl.index)):
    for j in range (len(dfReady.index)):
                    if dfCl.loc[i, 'Neighbourhood'] == dfReady.loc[j, 'Neighbourhood']:
                        latlist.append(dfReady.loc[j, 'Nlat'])
                        lonlist.append(dfReady.loc[j, 'Nlon'])
                        
                        break

                        

In [26]:

dfNLAT = pd.DataFrame(latlist, columns = ['Neighborhood Latitude'])

In [27]:
dfNLON = pd.DataFrame(lonlist, columns = ['Neighborhood Longitude'])

In [28]:
dfLL= dfNLON.join(dfNLAT)

In [29]:
dfLL

,Neighborhood Longitude,Neighborhood Latitude
0,-79.262029,43.794200
1,-79.543484,43.602414
2,-79.442259,43.754328
3,-79.385975,43.786947
4,-79.419750,43.733283
...,...,...
84,-79.408493,43.770120
85,-79.442259,43.782736
86,-79.216917,43.770992
87,-79.318389,43.695344


In [30]:
dfForMap = dfCl.join(dfLL)
dfForMap

,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,...,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood Longitude,Neighborhood Latitude
0,Agincourt,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,-79.262029,43.794200
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.543484,43.602414
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-79.442259,43.754328
3,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.385975,43.786947
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,-79.419750,43.733283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,"Willowdale, Willowdale East",0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,-79.408493,43.770120
85,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.442259,43.782736
86,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.216917,43.770992
87,Woodbine Heights,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.318389,43.695344


In [31]:
from sklearn.cluster import KMeans

In [32]:
k = 5
dfClus = dfCl.drop('Neighbourhood', axis = 1)

In [33]:
Clusted = KMeans(k).fit(dfClus)

In [34]:
Clusted.labels_[0:10]

array([3, 1, 1, 3, 0, 3, 1, 4, 3, 3], dtype=int32)

,cluster labels,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,...,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood Longitude,Neighborhood Latitude
0,3,Agincourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.262029,43.794200
1,1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.543484,43.602414
2,1,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-79.442259,43.754328
3,3,Bayview Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.385975,43.786947
4,0,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,-79.419750,43.733283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3,"Willowdale, Willowdale East",0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,-79.408493,43.770120
85,1,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.442259,43.782736
86,3,Woburn,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.216917,43.770992
87,3,Woodbine Heights,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.318389,43.695344


In [35]:
dfCl.insert(0,'cluster labels', Clusted.labels_)

In [36]:
dfNN= dfCl.join(dfLL)



In [37]:
dfNN

,cluster labels,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,...,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood Longitude,Neighborhood Latitude
0,3,Agincourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.262029,43.794200
1,1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.543484,43.602414
2,1,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-79.442259,43.754328
3,3,Bayview Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.385975,43.786947
4,0,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,-79.419750,43.733283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3,"Willowdale, Willowdale East",0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,-79.408493,43.770120
85,1,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.442259,43.782736
86,3,Woburn,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.216917,43.770992
87,3,Woodbine Heights,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.318389,43.695344


In [42]:
dfCl

,cluster labels,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,...,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood Longitude,Neighborhood Latitude
0,3,Agincourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.262029,43.794200
1,1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.543484,43.602414
2,1,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-79.442259,43.754328
3,3,Bayview Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.385975,43.786947
4,0,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,-79.419750,43.733283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3,"Willowdale, Willowdale East",0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,-79.408493,43.770120
85,1,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.442259,43.782736
86,3,Woburn,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.216917,43.770992
87,3,Woodbine Heights,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.318389,43.695344


spaced =[]
for i in range(len(dfCl.index)):
    for k in range(len(M.index)):
        if 

In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [39]:
dfCl = dfNN


In [40]:
dfCl

,cluster labels,Neighbourhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,...,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood Longitude,Neighborhood Latitude
0,3,Agincourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.262029,43.794200
1,1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.543484,43.602414
2,1,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,-79.442259,43.754328
3,3,Bayview Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.385975,43.786947
4,0,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,-79.419750,43.733283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3,"Willowdale, Willowdale East",0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,-79.408493,43.770120
85,1,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.442259,43.782736
86,3,Woburn,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.216917,43.770992
87,3,Woodbine Heights,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-79.318389,43.695344


In [41]:
mapFinal = folium.Map(location = [43.7, -79.5], zoom_start = 10)
x = np.arange(k)
y = [i + x + (i*x)**2 for i in range(k)]
colrs = cm.rainbow(np.linspace(0,1,len(y)))
rainbow = [colors.rgb2hex(i) for i in colrs ]

marks = []
for lat, lon, cluster, neighborhood in zip(dfCl['Neighborhood Latitude'], dfCl['Neighborhood Longitude'], dfCl['cluster labels'], dfCl['Neighbourhood']):
    
    
    
    label = folium.Popup(str(neighborhood) + ' Cluster: ' + str(cluster))
    folium.CircleMarker(
    [lat,lon],radius = 5, 
    color = rainbow[cluster - 1],
    popup = label,
    fill = True,
    fill_color = rainbow[cluster-1],
    fill_opacity = 0.5).add_to(mapFinal)
    
    

mapFinal    
    